# 02. Bedrock Client 테스트

AWS Bedrock Knowledge Base와 RetrieveAndGenerate API를 테스트합니다.

## 1. 환경 설정 및 라이브러리 import

In [ ]:
import os
import sys
import boto3
import json
import time
from dotenv import load_dotenv
from botocore.exceptions import ClientError

# 환경 변수 로드
load_dotenv(override=True)

# AWS 클라이언트 초기화
region = os.getenv('AWS_REGION', 'us-west-2')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)  # RetrieveAndGenerate용
bedrock_agent_client = boto3.client('bedrock-agent', region_name=region)  # Ingestion Job용
s3_client = boto3.client('s3', region_name=region)

print(f"✅ AWS 클라이언트 초기화 완료 (Region: {region})")

## 2. 환경 변수 확인

In [ ]:
# .env 파일에서 설정 로드
kb_id = os.getenv('BEDROCK_KNOWLEDGE_BASE_ID')
data_source_id = os.getenv('BEDROCK_DATA_SOURCE_ID')
s3_bucket_name = os.getenv('S3_BUCKET_NAME')
model_id = os.getenv('BEDROCK_MODEL_ID')
account_id = os.getenv('AWS_ACCOUNT_ID')
region = os.getenv('AWS_REGION')
rerank_model_id = os.getenv('BEDROCK_RERANK_MODEL_ID', 'cohere.rerank-v3-5:0')

print("🔍 환경 변수 확인:")
print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {data_source_id}")
print(f"S3 Bucket: {s3_bucket_name}")
print(f"Model ID: {model_id}")
print(f"AWS_ACCOUNT_ID: {account_id}")
print(f"AWS_REGION: {region}")
print(f"RERANK_MODEL_ID: {rerank_model_id}")

if not all([kb_id, data_source_id, s3_bucket_name, model_id]):
    print("\n⚠️ 일부 환경 변수가 설정되지 않았습니다.")
    print("01_aws_setup.ipynb를 먼저 실행하세요.")
else:
    print("\n✅ 모든 환경 변수가 설정되었습니다.")

## 3. Bedrock Client 클래스 정의

In [ ]:
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockClient:
    """AWS Bedrock Knowledge Base와 상호작용하는 클라이언트"""
    
    def __init__(self, knowledge_base_id, model_id, region='us-west-2'):
        self.knowledge_base_id = knowledge_base_id
        self.model_id = model_id
        self.region = region

        # AWS 계정 ID 가져오기
        try:
            sts_client = boto3.client('sts', region_name=region)
            self.account_id = sts_client.get_caller_identity()['Account']
        except Exception as e:
            print(f"AWS 계정 ID 조회 실패: {e}")
            self.account_id = None
        
        # Bedrock 클라이언트 초기화
        self.bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)
        self.bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)
        
        logger.info(f"BedrockClient 초기화 완료 - KB ID: {knowledge_base_id}")
    
    def retrieve_and_generate(self, query, session_id=None):
        """Knowledge Base에서 검색하고 응답 생성"""
        try:
            logger.info(f"🔍 Knowledge Base 검색 시작: {query[:50]}....")
            
            # RetrieveAndGenerate API 호출 파라미터
            params = {
                'input': {
                    'text': query
                },
                'retrieveAndGenerateConfiguration': {
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': self.knowledge_base_id,
                        'modelArn': f'arn:aws:bedrock:{self.region}:{self.account_id}:inference-profile/us.{self.model_id}'
                    }
                }
            }
            
            # 세션 ID가 있으면 추가
            if session_id:
                params['sessionId'] = session_id
            
            # API 호출
            response = self.bedrock_agent_runtime_client.retrieve_and_generate(**params)
            
            # 응답 파싱
            answer = response['output']['text']
            session_id = response.get('sessionId')
            citations = response.get('citations', [])
            
            # 소스 문서 정보 추출
            sources = []
            for citation in citations:
                for reference in citation.get('retrievedReferences', []):
                    location = reference.get('location', {})
                    s3_location = location.get('s3Location', {})
                    
                    source_info = {
                        'content': reference.get('content', {}).get('text', ''),
                        'filename': s3_location.get('uri', '').split('/')[-1] if s3_location.get('uri') else 'Unknown',
                        'uri': s3_location.get('uri', ''),
                        'confidence': reference.get('metadata', {}).get('confidence', 0)
                    }
                    sources.append(source_info)
            
            logger.info(f"✅ 검색 완료: {len(sources)}개 소스 발견")
            
            return {
                'answer': answer,
                'sources': sources,
                'session_id': session_id,
                'citations': citations
            }
            
        except ClientError as e:
            error_code = e.response['Error']['Code']
            error_message = e.response['Error']['Message']
            logger.error(f"❌ Bedrock API 오류 [{error_code}]: {error_message}")
            
            return {
                'answer': f"죄송합니다. 검색 중 오류가 발생했습니다: {error_message[:100]}...",
                'sources': [],
                'session_id': session_id,
                'error': True,
                'error_code': error_code
            }
            
        except Exception as e:
            logger.error(f"❌ 예상치 못한 오류: {str(e)}")
            
            return {
                'answer': f"죄송합니다. 시스템 오류가 발생했습니다: {str(e)[:100]}...",
                'sources': [],
                'session_id': session_id,
                'error': True,
                'error_code': 'SystemError'
            }

# BedrockClient 인스턴스 생성
if kb_id and model_id:
    bedrock_client = BedrockClient(kb_id, model_id, region)
    print("✅ BedrockClient 인스턴스 생성 완료")
else:
    print("⚠️ Knowledge Base ID 또는 Model ID가 없어 클라이언트를 생성할 수 없습니다.")
    bedrock_client = None

## 4. 기본 연결 테스트

In [ ]:
# Knowledge Base 연결 테스트
if bedrock_client:
    print("🔍 Knowledge Base 연결 테스트...")
    
    test_query = "안녕하세요. 연결 테스트입니다."
    result = bedrock_client.retrieve_and_generate(test_query)
    
    print(f"📝 응답: {result['answer'][:100]}...")
    print(f"📚 소스 개수: {len(result['sources'])}개")
    
    if result.get('error'):
        print(f"⚠️ 오류 발생: {result.get('error_code')}")
    else:
        print("✅ Knowledge Base 연결 성공")
else:
    print("⚠️ BedrockClient가 초기화되지 않아 테스트를 건너뜁니다.")

## 5. 테스트 문서 업로드 및 Knowledge Base 동기화

In [ ]:
import os
import time
from pathlib import Path

def upload_test_documents():
    """tests 디렉토리의 RAG 관련 문서들을 S3에 업로드"""
    
    print("📄 테스트 문서 업로드 시작...")
    
    # 업로드할 파일들 정의
    test_files = [
        {
            'local_path': 'tests/aws_rag_guide.md',
            's3_key': 'documents/aws_rag_guide.md',
            'content_type': 'text/markdown'
        }
    ]
    
    uploaded_files = []
    
    for file_info in test_files:
        local_path = file_info['local_path']
        s3_key = file_info['s3_key']
        content_type = file_info['content_type']
        
        # 파일 존재 확인
        if not os.path.exists(local_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {local_path}")
            continue
            
        try:
            # 파일 읽기
            with open(local_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # S3에 업로드
            s3_client.put_object(
                Bucket=s3_bucket_name,
                Key=s3_key,
                Body=content.encode('utf-8'),
                ContentType=content_type
            )
            
            uploaded_files.append(os.path.basename(local_path))
            print(f"✅ {os.path.basename(local_path)} 업로드 완료")
            
        except Exception as e:
            print(f"❌ {local_path} 업로드 실패: {e}")
    
    print(f"\n📊 총 {len(uploaded_files)}개 테스트 문서 업로드 완료")
    return uploaded_files

# upload_test_documents 함수 정의 후
print(f"함수 정의 후: {bedrock_agent_client._service_model.service_name}")

def sync_knowledge_base(kb_id, data_source_id):
    """Knowledge Base와 S3 데이터 소스 동기화"""
    try:
        print("\n🔄 Knowledge Base 동기화 시작...")
        
        # Ingestion Job 시작
        response = bedrock_agent_client.start_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=data_source_id,
            description='Test documents from tests directory'
        )
        
        job_id = response['ingestionJob']['ingestionJobId']
        print(f"📋 Ingestion Job ID: {job_id}")
        
        # Job 상태 모니터링
        print("⏳ 동기화 진행 중...")
        
        max_wait_time = 300  # 최대 5분 대기
        start_time = time.time()
        
        while time.time() - start_time < max_wait_time:
            job_status = bedrock_agent_client.get_ingestion_job(
                knowledgeBaseId=kb_id,
                dataSourceId=data_source_id,
                ingestionJobId=job_id
            )
            
            status = job_status['ingestionJob']['status']
            print(f"📊 상태: {status}")
            
            if status == 'COMPLETE':
                print("✅ Knowledge Base 동기화 완료!")
                return True
                
            elif status == 'FAILED':
                print("❌ Knowledge Base 동기화 실패")
                failure_reasons = job_status['ingestionJob'].get('failureReasons', [])
                for reason in failure_reasons:
                    print(f"   - {reason}")
                return False
                
            elif status in ['IN_PROGRESS', 'STARTING']:
                time.sleep(15)  # 15초 대기
            else:
                print(f"⚠️ 예상치 못한 상태: {status}")
                time.sleep(10)
        
        print("⏰ 동기화 대기 시간 초과")
        return False
                
    except Exception as e:
        print(f"❌ 동기화 오류: {e}")
        return False

# sync_knowledge_base 함수 정의 후  
print(f"sync 함수 정의 후: {bedrock_agent_client._service_model.service_name}")

# 디버깅: 클라이언트 타입 확인
print(f"bedrock_agent_client 타입: {type(bedrock_agent_client)}")
print(f"bedrock_agent_client 서비스: {bedrock_agent_client._service_model.service_name}")

# 실행 코드 시작 전
print(f"실행 코드 시작 전: {bedrock_agent_client._service_model.service_name}")

# 실행 코드
if not all([kb_id, data_source_id, s3_bucket_name]):
    print("❌ 필요한 환경 변수가 설정되지 않았습니다.")
    print("01_aws_setup.ipynb를 먼저 실행하세요.")
else:
    # 테스트 문서 업로드
    uploaded_files = upload_test_documents()
    
    if uploaded_files:
        # Knowledge Base 동기화
        sync_success = sync_knowledge_base(kb_id, data_source_id)
        
        if sync_success:
            print("\n🎉 테스트 문서가 Knowledge Base에 성공적으로 추가되었습니다!")
        else:
            print("\n⚠️ 동기화에 문제가 있었습니다.")
    else:
        print("\n❌ 업로드된 파일이 없어 동기화를 건너뜁니다.")



## 6. retrieve_and_generate 함수 정의

In [ ]:
def retrieve_and_generate(query, temperature=0.1, max_tokens=2048):
    """Knowledge Base를 사용한 RAG 검색 및 답변 생성 (03 project 검증된 로직)"""
    
    # 청크 설정 (03 project와 동일)
    SOURCE_CHUNKS = 50
    RERANK_CHUNKS = 5
    
    if not bedrock_agent_runtime_client or not account_id:
        return {
            'answer': "클라이언트 초기화 실패",
            'citations': [],
            'success': False
        }

    try:
        # ARN 생성 (02 노트북 변수명 사용)
        model_arn = f'arn:aws:bedrock:{region}:{account_id}:inference-profile/us.{model_id}'
        rerank_arn = f'arn:aws:bedrock:{region}::foundation-model/{rerank_model_id}'
        
        print(f"Model ARN: {model_arn}")
        
        request_params = {
            'input': {'text': query},
            'retrieveAndGenerateConfiguration': {
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,  # 02 노트북 변수명
                    'modelArn': model_arn,
                    'retrievalConfiguration': {
                        'vectorSearchConfiguration': {
                            'numberOfResults': SOURCE_CHUNKS,  # 50개 청크
                            'overrideSearchType': 'HYBRID',
                            'rerankingConfiguration': {
                                'type': 'BEDROCK_RERANKING_MODEL',
                                'bedrockRerankingConfiguration': {
                                    'modelConfiguration': {
                                        'modelArn': rerank_arn
                                    },
                                    'numberOfRerankedResults': RERANK_CHUNKS  # 5개로 축소
                                }
                            }
                        }
                    },
                    'generationConfiguration': {
                        'inferenceConfig': {
                            'textInferenceConfig': {
                                'maxTokens': max_tokens,
                                'temperature': temperature,
                                'topP': 0.9
                            }
                        }
                    }
                }
            }
        }

        # 02 노트북 클라이언트 사용
        response = bedrock_agent_runtime_client.retrieve_and_generate(**request_params)

        # 실제 사용된 청크 수 확인
        actual_chunks_used = len(response.get('citations', []))
        
        return {
            'answer': response['output']['text'],
            'citations': response.get('citations', []),
            'success': True,
            'actual_chunks_used': actual_chunks_used
        }

    except Exception as e:
        return {
            'answer': f"오류가 발생했습니다: {str(e)}",
            'citations': [],
            'success': False
        }

print("✅ retrieve_and_generate 함수 정의 완료")


## 7. RetrieveAndGenerate API 테스트

In [ ]:
# 청크 설정 (03 프로젝트와 동일)
SOURCE_CHUNKS = 50  # 초기 검색 청크 수
RERANK_CHUNKS = 5   # 최종 선택 청크 수

# 테스트 질문
test_queries = "80mm 압력 감소 밸브 설치에 필요한 인력은?"

print(f"🧪 테스트 질문: {test_query}")
print(f"설정: {SOURCE_CHUNKS}개 검색 → {RERANK_CHUNKS}개 선별")
print("-" * 50)

# RAG 실행
result = retrieve_and_generate(test_query)

if result['success']:
    print("✅ 테스트 성공!")
    print(f"\n답변: {result['answer'][:200]}...")
    print(f"\n사용된 청크: {result.get('actual_chunks_used', 0)}개")
    print(f"참고 문서: {len(result['citations'])}개")
else:
    print("❌ 테스트 실패")
    print(f"오류: {result['answer']}")

print("\n✅ 함수 테스트 완료!")


## 8. Streamlit 웹 앱 실행

In [ ]:
# Streamlit 설정 파일 생성 (Jupyter 환경용)
import os
import subprocess
import time

# Streamlit 설정 디렉토리 생성
os.makedirs(os.path.expanduser('~/.streamlit'), exist_ok=True)

# 설정 파일 생성
config_content = """[server]
enableXsrfProtection = false
enableCORS = false
port = 8501
headless = true

[browser]
gatherUsageStats = false
"""

with open(os.path.expanduser('~/.streamlit/config.toml'), 'w') as f:
    f.write(config_content)

print("✅ Streamlit 설정 파일이 생성되었습니다.")

# Streamlit 앱 실행
def run_streamlit_app():
    """Streamlit 앱 실행"""
    if not os.path.exists('app.py'):
        print("❌ app.py 파일이 없습니다.")
        return None
    
    print("🚀 Streamlit 앱을 시작합니다...")
    print("📱 브라우저에서 http://localhost:8501 로 접속하세요")
    
    try:
        # Streamlit 실행
        process = subprocess.Popen([
            'streamlit', 'run', 'app.py'
        ])
        
        print("✅ Streamlit 서버가 시작되었습니다!")
        print("🌐 URL: http://localhost:8501")
        print("⏹️ 중지하려면 Kernel을 재시작하세요")
        
        return process
        
    except Exception as e:
        print(f"❌ Streamlit 실행 실패: {e}")
        return None

# 실행
streamlit_process = run_streamlit_app()

print("\n📋 주요 기능:")
print("✅ 멀티모달 RAG 검색")
print("✅ Rerank 기반 고품질 답변")
print("✅ 실시간 채팅 인터페이스")
print("✅ 파일 업로드 (PDF, 텍스트, 이미지)")
print("✅ 소스 문서 추적 및 표시")
print("✅ 세션 기반 대화 관리")